In [1]:
import requests
from bs4 import BeautifulSoup
from urllib import request

In [19]:
page = 1

comp_list=[]
date_list=[]


total_page = 233

while True:
    url = 'https://finance.naver.com/research/debenture_list.nhn?' 
    
    
    params = {
        'page' : page
    }

    
    resp = requests.get(url, params=params)
    soup = BeautifulSoup(resp.content, 'html.parser')

    #링크
    td_link = soup.find('table', class_='type_1').find_all('td', class_='file')
    td_date = soup.find('table', class_='type_1').find_all('td', class_='date')
    tr_comp = soup.find('table', class_= 'type_1').find_all('tr')

    #증권사 이름 추출
    
    for tr in tr_comp:
        try:
            comp=tr.find_all('td')[1].text
            comp_list.append(comp)
            print(f'{comp}', end = ' ')
        except:
            continue
    
    #날짜 추출
    
    for idx, td in enumerate(td_date):
        if idx % 2 == 0:
            date_list.append(td.text)
            print(f'{td.text} 읽음', end=' ')

    
        
    if page>=total_page : 
        break
        
    page+=1 
    print()

한화투자증권 교보증권 교보증권 하나증권 다올투자증권 유진투자증권 신한투자증권 키움증권 유안타증권 하이투자증권 IBK투자증권 메리츠증권 교보증권 하나증권 하나증권 한화투자증권 다올투자증권 유진투자증권 키움증권 유안타증권 신한투자증권 신한투자증권 교보증권 다올투자증권 유진투자증권 유안타증권 신한투자증권 한화투자증권 한화투자증권 교보증권 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.19 읽음 24.03.19 읽음 24.03.19 읽음 24.03.19 읽음 24.03.19 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 
신한투자증권 다올투자증권 하이투자증권 유안타증권 키움증권 유진투자증권 IBK투자증권 신한투자증권 신한투자증권 한화투자증권 교보증권 다올투자증권 유진투자증권 유안타증권 키움증권 교보증권 다올투자증권 유안타증권 유진투자증권 메리츠증권 신한투자증권 교보증권 하나증권 하나증권 유안타증권 유진투자증권 다올투자증권 신한투자증권 유안타증권 교보증권 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.14 읽음 24.03.14 읽음 24.03.14 읽음 24.03.14 읽음 24.03.14 읽음 24.03.13 읽음 24.03.13 읽음

In [9]:
def pdfToText(fname):

    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO
    
    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(f'rawReport/{fname}', 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ''
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages, password = password,
                                      caching = caching, check_extractable = True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        fp.close()
        device.close()
        retstr.close()

        return text
    except Exception as e:
        return str(e)

In [10]:
len(comp_list)

6970

In [11]:
len(date_list)

6970

In [25]:
if len(comp_list) != len(date_list):
    raise ValueError("comp_list와 date_list의 길이가 다릅니다.")


In [ ]:
import pandas as pd

text_list=[]
for idx, comp in enumerate(comp_list):
    
    page = (idx // 30)+1
    p_no = (idx % 30)+1
    
    text=pdfToText(f'{page}_{p_no}.pdf')
    
    print(f'{page}_{p_no}.pdf 텍스트 추출...')
    text=text.replace('\n','')
    text=text.replace('\r','')
    text=text.replace('\t','')
    
    text_list.append(text)
    
    if (idx+1) % 30 == 0:
        dict_report ={
            '증권사' : comp_list[idx-29:idx+1],
            '날짜' : date_list[idx-29:idx+1],
            '텍스트' : text_list
        }
        
        df = pd.DataFrame(dict_report)
        df.to_csv(f'bond_report page_{page}.csv', index=False)
        
        text_list = []


rest = len(text_list)

dict_report ={
            '증권사' : comp_list[len(comp_list)-rest:],
            '날짜' : date_list[len(date_list)-rest:],
            '텍스트' : text_list
            }
        
df = pd.DataFrame(dict_report)
df.to_csv(f'bond_report page_{page}.csv', index=False)

In [27]:
import pandas as pd

def process_and_save_data(comp_list, date_list, text_list, page):
    dict_report ={
        '증권사': comp_list,
        '날짜': date_list,
        '텍스트': text_list
    }
    df = pd.DataFrame(dict_report)
    df.to_csv(f'bond_report_page_{page}.csv', index=False)

comp_per_page = 30
total_page = (len(comp_list) + comp_per_page - 1) // comp_per_page

for page in range(1, total_page + 1):
    comp_slice = comp_list[(page - 1) * comp_per_page: page * comp_per_page]
    date_slice = date_list[(page - 1) * comp_per_page: page * comp_per_page]
    text_slice = []
    for idx, comp in enumerate(comp_slice):
        p_no = (page - 1) * comp_per_page + idx + 1
        text = pdfToText(f'{page}_{p_no}.pdf')
        text = text.replace('\n', '').replace('\r', '').replace('\t', '')
        text_slice.append(text)

    process_and_save_data(comp_slice, date_slice, text_slice, page)



In [24]:
import csv


csv_filename = 'debenture_data.csv'


with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    
    writer.writerow(['증권사', '날짜', '제목'])
    
    
    for comp, date, title in zip(comp_list, date_list, titles):
        writer.writerow([comp, date, title])

print(f'Data saved to {csv_filename}')


Data saved to debenture_data.csv


In [52]:
import requests
from bs4 import BeautifulSoup

url = "https://finance.naver.com/research/debenture_read.naver?nid=7526&page=4"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


target_text = soup.find_all('div', style="width:555px;height:100% clear:both; text-align: justify; overflow-x: auto;padding: 20px 0pt 30px;font-size:9pt;line-height:160%; color:#000000;")


for text in target_text:
    print(text.get_text().strip())


Reverse OT (역 오퍼레이션 트위스트)지난 주말에 크리스토퍼 월러 연준 이사는 연준의 단기채 보유 비중이 더 확대되길 바란다고 밝혔다. 그는 글로벌 금융위기 이전에 재정증권이 Treasury holdings에서 약 3분의 1 비중을 차지했지만 지금은 5% 미만, Total securities holdings 대비로는 3% 수준까지 줄었다고 언급했다. 단기채 비중이 늘어나면 FOMC의 정책금리 인상/인하 결정에 따라 연준의 수익/비용도 함께 등락할 수 있기 때문이라고 덧붙였다. 그리고 향후 QE (양적완화) 프로그램에도 도움이 될 수 있다고 했는데, 이는 결국 매입한 재정증권을 자동 만기 회수하는 방식을 의미한다. 그는 QE가 영구적인 유동성 주입으로 간주되면 인플레이션을 야기할 수 있는 만큼 QE 이후 신뢰할 수 있는 QT (양적긴축)가 이어지는 것이 중요하다고 밝혔다. 나아가 MBS 보유량을 ‘제로’로 감소시키는 것도 보고 싶다고 발언했다. 물론 이러한 B/S 조정 과정이 정책금리 결정과 독립적임을 거듭 강조했다. 하지만 월러 이사 발언 배경을 먼저 살펴볼 필요가 있다. 현재 연준 포트폴리오는 자산/부채 미스매치로 적자를 기록하고 있다. 자산은 과거 QE을 통해 매입한 낮은 금리의 장기채가 대부분을 차지하고 있는 반면, 부채는 높은 금리일 뿐만 아니라 만기도 짧기 때문이다 (역레포/지준).


In [ ]:
#contentarea_left > div.box_type_m.box_type_m3 > table > tbody > tr:nth-child(3) > td > div:nth-child(1) > p:nth-child(2)

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

# 다운로드할 PDF 파일을 저장할 디렉토리를 지정합니다.
download_dir = 'pdf_files'
os.makedirs(download_dir, exist_ok=True)

# 웹 페이지 URL을 지정합니다.
url = 'https://finance.naver.com/research/debenture_list.naver'

# 웹 페이지의 HTML을 가져옵니다.
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# HTML에서 PDF 파일에 대한 링크를 찾습니다.
pdf_links = []
for link in soup.find_all('a'):
    href = link.get('href')
    if href and href.endswith('.pdf'):
        pdf_links.append(href)

# PDF 파일을 다운로드합니다.
for pdf_link in pdf_links:
    pdf_url = pdf_link
    # PDF 파일 이름을 추출합니다.
    pdf_filename = os.path.join(download_dir, pdf_url.split('/')[-1])
    # PDF 파일을 다운로드합니다.
    with open(pdf_filename, 'wb') as f:
        print(f'Downloading {pdf_filename}...')
        response = requests.get(pdf_url)
        f.write(response.content)

print('Download completed.')


In [9]:
pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
import os
import fitz


directory_path = r'C:\Users\SDA17\Desktop\workspace\BOK_project_TEAM3\j_craw\pdf_files'

for filename in os.listdir(directory_path):
    
    file_path = os.path.join(directory_path, filename)
    
    
    if filename.endswith('.pdf'):
        print(f'Extracting text from {filename}...')
        
        
        with fitz.open(file_path) as doc:
            
            for page in doc:
                text = page.get_text()
                
                print(text)

print('Text extraction completed.')


In [ ]:
import os
import fitz

directory_path = r'C:\Users\SDA17\Desktop\workspace\BOK_project_TEAM3\j_craw\pdf_files'

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    
    if filename.endswith('.pdf'):
        print(f'Extracting text from {filename}...')
        
        with fitz.open(file_path) as doc:
            for page in doc:
                text = page.get_text()
                print(text)  # 각 페이지에서 추출된 텍스트를 출력합니다.

print('Text extraction completed.')

In [2]:
import os
import fitz
import re

directory_path = r'C:\Users\SDA17\Desktop\workspace\BOK_project_TEAM3\j_craw\pdf_files'
text_by_date = {}  # 날짜별로 텍스트를 저장할 딕셔너리

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    
    if filename.endswith('.pdf'):
        date_match = re.search(r'(\d{4}\-\d{2}\-\d{2})', filename)  # 파일 이름에서 날짜 정보 추출 (예: 2024-03-22)
        if date_match:
            date = date_match.group(1)
            print(f'Extracting text from {filename} (Date: {date})...')
            
            if date not in text_by_date:
                text_by_date[date] = []  
            
            with fitz.open(file_path) as doc:
                for page in doc:
                    text = page.get_text()
                    text_by_date[date].append(text)  

.
for date, texts in text_by_date.items():
    print(f'Date: {date}')
    for text in texts:
        print(text)

print('Text extraction and grouping completed.')


Text extraction and grouping completed.
